In [10]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('/Users/inejj/PycharmProjects/Projectjes/HelloFoodFreak/meal_delivery/train_GzS76OK/train.csv')

FileNotFoundError: [Errno 2] File /Users/inejj/PycharmProjects/Projectjes/HelloFoodFreak/meal_delivery/train_GzS76OK/train.csv does not exist: '/Users/inejj/PycharmProjects/Projectjes/HelloFoodFreak/meal_delivery/train_GzS76OK/train.csv'